In [134]:
from Fitting import Fitting
import json

In [135]:
with open("ASHRAE_CR3-1.json") as file:
    elbow = json.load(file)

In [136]:
fit1 = Fitting(elbow)

In [137]:
fit1.fittype

'ELBOW'

In [138]:
mycondition={}
mycondition["H_i"] = 200
mycondition["W_i"] = 200
mycondition["r"] = 200
mycondition["th"] = 45

In [140]:
print(fit1.get_Loss(mycondition))

0.126


In [24]:
from enum import Enum
from enum import auto
from Fitting import Fitting
import math

In [157]:
class Duct(object):
    
    class REF(Enum):
        FLOW = auto()
        SIZE = auto()
        LOSS = auto()
        VELOCITY = auto()
    
    FLOW = REF.FLOW
    SIZE = REF.SIZE
    LOSS = REF.LOSS
    VELOCITY = REF.VELOCITY
    
    class DuctType(Enum):
        RECTANGULAR = auto()
        ROUND = auto()
        OVAL = auto()
        
    def __init__(self, dtype:DuctType=DuctType.ROUND,
                 diam:float=300.,
                 size:list=[300., 300.],
                 length:float=1.22,
                 flow:float=100.,
                 e:float=0.09):
        self.name = "duct"
        self.__fittings = []
        self.dtype = dtype
        try:
            if (len(size) == 2):
                self.size = size
            else:
                self.size = [300, 300]
        except TypeError as e:
            self.size = [diam, 0.]  
        self.__flow = flow
        self.roughness = e
        
    @property
    def name(self):
        return self.__name
    
    @name.setter
    def name(self, value):
        self.__name = value
    
    def add_fitting(self, fitt:Fitting, cond:dict):
        self.__fittings.append((fitt, cond))
    
    def remove_fitting(self, i:int):
        del(self.__fittings[i])
    
    def clear_fittings(self, i:int):
        self.__fittings = []
    
    def is_width_fixed(self):
        return self.__is_fixed
    
    def fix_width(self, yn:bool = True):
        self.__is_fixed = yn
    
    def __gwi(self) -> int:
        if is_width_fixed():
            return 1
        else:
            return 0
    
    @property
    def dtype(self):
        return self.__dtype
    
    @dtype.setter
    def dtype(self, value):
        try:
            self.__dtype = value
        except:
            print("Bandera")
            self.__dtype = value
    @property
    def flow(self):
        return self.__flow
    
    @flow.setter
    def flow(self, flow:float, ref:REF):
        if (flow < 0 or flow == self.__flow):
            return
        if (ref == LOSS):
            refval = self.loss_rate
            self.flow(flow, REF.SIZE)
            e = 1.0
            count = 0
            rate = 100.
            index = self.__gwi()
            while (e > 0.0001 or count < 1000):
                if (refval < self.loss_rate):
                    self.__size[index] += rate
                    if (refval > self.loss_rate):
                        self.__size[index] -= rate
                        rate /= 10.
                elif (refval > self.loss_rate):
                    self.__size[index] -= rate
                    if (refval < self.loss_rate or self.__size[index] < 0.):
                        a_size[index] += rate;
                        rate /= 10.
                count += 1
                if count == 999:
                    print("Stopped by counter")
                e = abs(self.loss_rate - refval)
        elif (ref == VELOCITY):
            refval = self.fluid_velocity
            self.flow(flow, REF.SIZE)
            e = 1.0
            count = 0
            rate = 100.
            index = self.__gwi()
            while (e > 0.0001 or count < 1000):
                if (refval < self.fluid_velocity):
                    self.__size[index] += rate
                    if (refval > self.fluid_velocity()):
                        self.__size[index] -= rate
                        rate /= 10.
                elif (refval > self.fluid_velocity):
                    self.__size[index] -= rate
                    if (refval < self.fluid_velocity() or self.__size[index] < 0.):
                        a_size[index] += rate
                        rate /= 10.
                count += 1
                if count == 999:
                    print("Stopped by counter")
                e = abs(self.loss_rate - refval)
        else: ## ref == SIZE or ref == FLOW:
            self.__flow = flow
    
    @property
    def size(self):
        return self.__size
    
    @size.setter
    def size(self, value):
        if (len(value) == 2 and value[0] > 0. and value[1] > 0):
            self.__size = value
    
    def set_size(self, size:list = [], diam:float = 0, dtype:DuctType = DuctType.ROUND):
        if (dtype == DuctType.RECTANGULAR):
            pass
            ## Pending to define
        elif (dtype == DuctType.ROUND):
            if diam > 0:
                setDe(diam)
            ## Pending to define
        else: ## dtype == OVAL
            pass
            ## Pending to define
    
    @property
    def roughness(self):
        return self.__roughness
    
    @roughness.setter
    def roughness(self, value):
        if (value > 0.):
            self.__roughness = value
    
    def get_area(self):
        if (self.__dtype == self.DuctType.RECTANGULAR):
            return self.__size[0] * self.__size[1] / 1.0e6
        elif (self.__dtype == self.DuctType.ROUND):
            return math.pi * self.__size[0] * self.__size[0] / 4.0e6
        else: ## self.__dtype == OVAL
            return math.pi * self.__size[1] * self.__size[1] / 4.0e6\
              + self.__size[1] * (self.size[0] - self.size[1]) / 1.0e6
    
    def get_fluid_area(self):
        return math.pi * self.get_De() ** 2.0 / 4.0e6
    
    def get_perim(self):
        if (self.__dtype == self.DuctType.RECTANGULAR):
            return 2.0 * (self.__size[0] + self.__size[1])
        elif (self.__dtype == self.DuctType.ROUND):
            return math.pi * self.__size[0]
        else: ## self.__dtype == OVAL
            return math.pi * self.__size[1] + 2 * (self.size[0] - self.size[1])
    
    def get_hid_diam(self):
        if (self.dtype == self.DuctType.ROUND):
            return self.__size[0]
        else:
            return 4.0e6 * self.get_area() / self.get_perim()
        
    def get_De(self):
        if (self.dtype == self.DuctType.RECTANGULAR):
            return 1.3 * math.pow(self.__size[0] * self.__size[1], 0.625)\
              / math.pow(self.__size[0] + self.__size[1], 0.25)
        elif (self.dtype == self.DuctType.ROUND):
            return self.__size[0]
        else: ## self.__dtype == self.DuctType.OVAL
            return 1.55 * Math.pow(get_area(), 0.625) / Math.pow(get_perim(), 0.25)
    
    def set_De(self, d:float, ref:REF):
        if (d < 0.):
            return
        if (ref == self.REF.FLOW):
            pass
            ##Pending to define
        elif (ref == self.REF.LOSS):
            pass
            ##Pending to define
        elif (ref == self.REF.VELOCITY):
            pass
            ##Pending to define
        else: ## ref == self.REF.SIZE
            pass
        
    def get_velocity(self):
        return self.__flow / self.get_area() / 1000.
    
    def get_fluid_vel(self):
        return self.__flow / self.get_fluid_area() / 1000.
    
    def set_fluid_vel(self, vel:float, ref:REF):
        if (v < 0.):
            return
        if (ref == self.DuctType.FLOW):
            pass
            ## Pending to define
        elif (ref == self.DuctType.LOSS):
            pass
            ## Pending to define
        elif (ref == self.DuctType.SIZE):
            pass
            ## Pending to define
        else: ## ref == self.DuctType.VELOCITY
            pass
        
    def get_vel_pressure(self):
        return 0.602 * self.get_fluid_vel() ** 2.
    
    def get_ffactor(self):
        f = self.roughness / self.get_hid_diam()
        e = 10.
        count = 999
        while (e > 0.0001 or count < 999):
            f = self.FColebrook(f)
            e = abs(f - self.FColebrook(f))
            count += 1
            if (count == 999):
                print("Finished by count")
        return f
    def set_loss_rate(self, loss:float, ref:REF):
        if (loss < 0.0 or loss == self.get_loss()):
            return
        rate = 100.
        e = 100.
        count = 0
        if (ref == self.REF.FLOW):
            while (e > 0.00001 or count < 999):
                if (loss < self.get_loss()):
                    self.set_De(self.get_De() + rate)
                    if (loss > self.get_loss()):
                        self.set_De(self.get_De() - rate)
                        rate /= 10.;
                else:
                    self.set_De(get_De() - rate)
                    if (loss < self.get_loss() or get_De() - rate < 0.):
                        rate /= 10.
                count += 1
                if (count == 999):
                    print("Stopped by counter")
                e = abs(loss - self.get_loss())
        elif (ref == self.REF.VELOCITY):
            pass
            # Pending to define
        elif (rel == self.REF.SIZE):
            while (e > 0.00001):
                if (loss > get_loss()):
                    self.set_flow(self.get_flow() + rate, self.REF.SIZE)
                    if (loss < self.get_loss()):
                        self.set_flow(self.get_flow() + rate, self.REF.SIZE)
                elif (loss < self.get_loss()):
                    self.set_flow(self.get_flow() - rate, self.REF.SIZE)
                    if (loss > self.get_loss() or self.get_flow() - rate < 0.):
                        self.set_flow(self.get_flow() + rate, self.REF.SIZE)
        else:  # rel == self.REF.LOSS
            return
        
    def get_loss_rate(self):
        return 1000. * self.get_ffactor() / self.get_De() * self.get_vel_pressure()
    
    def get_loss(self, length, output_type:int = 0):
        if (output_type > 2):
            output_type = 0
        Fitloss = 0.0
        for i in range(len(self.__fittings)):
            if (self.__fittings[i][0].fittype == "UNCLASSIFIED" or \
              self.__fittings[i][0].fittype == "TEE" or \
              self.__fittings[i][0].fittype == "WYE"):
                Fitloss += self.__fittings[i][0].get_Loss(self.__fittings[i][1])[output_type]
            else:
                Fitloss += self.__fittings[i][0].get_Loss(self.__fittings[i][1])
        return self.get_loss_rate() * length + Fitloss * self.get_vel_pressure()
    
    def get_Reynolds(self):
        return 66.4 * self.get_hid_diam() * self.get_velocity()
    
    def FColebrook(self, f:float):
        return 1. / math.pow(-2.0 * math.log10(self.roughness / 3.7 / self.get_hid_diam()\
          + 2.51 / self.get_Reynolds() / math.sqrt(f)), 2.0)
    
    def __str__(self):
        result = ""
        result += "Flow (L/s):        {:2f}".format(self.flow) + '\n'
        if (self.dtype == self.DuctType.ROUND):
            result += "Diámetro (mm):     {:2f}".format(self.size[0]) + '\n'
        else:
            result += "Ancho (mm):        {:2f}".format(self.size[0]) + '\n'
            result += "Alto (mm):         {:2f}".format(self.size[1]) + '\n'
        result += "Eq. Diam. (mm):    {:2f}".format(self.get_De()) + '\n'
        result += "Hid. Diam (mm):    {:2f}".format(self.get_hid_diam()) + '\n'
        result += "Flow Area (m2):    {:2f}".format(self.get_fluid_area()) + '\n'
        result += "Fluid Vel. (m/s):  {:2f}".format(self.get_fluid_vel()) + '\n'
        result += "Reynolds (-):      {:2f}".format(self.get_Reynolds()) + '\n'
        result += "Friction fac. (-): {:2f}".format(self.get_ffactor()) + '\n'
        result += "Vel. Press. (Pa):  {:2f}".format(self.get_vel_pressure()) + '\n'
        result += "Head loss (Pa/m):  {:2f}".format(self.get_loss_rate())
        return result

In [158]:
duct1 = Duct(dtype=Duct.DuctType.RECTANGULAR)

In [159]:
print(duct1)

Flow (L/s):        100.000000
Ancho (mm):        300.000000
Alto (mm):         300.000000
Eq. Diam. (mm):    327.949602
Hid. Diam (mm):    300.000000
Flow Area (m2):    0.084470
Fluid Vel. (m/s):  1.183848
Reynolds (-):      22133.333333
Friction fac. (-): 0.025897
Vel. Press. (Pa):  0.843700
Head loss (Pa/m):  0.066623


In [160]:
duct1.add_fitting(fit1, mycondition)

In [161]:
print("Pérdida total (Pa): {:2f}".format(duct1.get_loss(length=1.22, output_type=0)))

Pérdida total (Pa): 0.187586


In [164]:
with open("ASHRAE_SR5-11.json") as file:
    wye = json.load(file)

In [165]:
fit2 = Fitting(wye)

In [166]:
print(fit2)

ASHRAE SR5-11


In [167]:
print(wye)

{'name': 'ASHRAE SR5-11', 'type': 'Tee', 'system': 'Suppy', 'description': 'Rectangular Main to Round Tap, Diverging', 'rel_xo': 'Q_o/Q_i', 'x_o': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 'rel_yo': 'A_o/A_i', 'y_o': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 'rel_x1': 'Q_1/Q_i', 'x_1': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 'rel_y1': 'A_1/A_i', 'y_1': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 'data_o': [[0.04, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.98, 0.04, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [3.48, 0.31, 0.04, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0], [7.55, 0.98, 0.18, 0.04, 0.02, 0.0, 0.0, 0.0, 0.0], [13.18, 2.03, 0.49, 0.13, 0.04, 0.0, 0.01, 0.0, 0.0], [20.38, 3.48, 0.98, 0.31, 0.1, 0.04, 0.02, 0.01, 0.0], [29.15, 5.32, 1.64, 0.6, 0.23, 0.09, 0.04, 0.02, 0.01], [39.48, 7.55, 2.47, 0.98, 0.42, 0.18, 0.08, 0.04, 0.02], [51.37, 10.17, 3.48, 1.46, 0.67, 0.31, 0.15, 0.07, 0.04]], 'data_1': [[1.58, 0.94, 0.83, 0.79, 0.77, 0.76, 0.76, 0.76, 0.75], [4.2, 1.58, 1.1, 0.9

In [168]:
cond2={}
cond2["A_i"] = 200
cond2["A_o"] = 200
cond2["A_1"] = 200
cond2["Q_i"] = 450
cond2["Q_o"] = 300
cond2["Q_1"] = 150

In [169]:
fit2.get_Loss(cond2)

(0.29333333333333333, 8.900000000000002)

In [170]:
duct1.add_fitting(fit2, cond2)

In [171]:
print("Pérdida total (Pa): {:2f}".format(duct1.get_loss(length=1.22, output_type=0)))

Pérdida total (Pa): 0.435072


In [172]:
print("Pérdida total (Pa): {:2f}".format(duct1.get_loss(length=1.22, output_type=1)))

Pérdida total (Pa): 7.696521
